In [2]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

BING_MAPS_API_KEY = os.environ["BING_MAPS_API_KEY"]

## Bing maps tools

In [3]:
from pydantic import BaseModel, Field
import requests


class LocalSearch(BaseModel):
    """ local search

    Returns a list of business entities centered around a location or a geographic region.
    For more information: https://learn.microsoft.com/en-us/bingmaps/rest-services/locations/local-search
    """
    user_location: str = Field(title="user location", description="Coordinates for the user's location on Earth.")
    query: str = Field(default="restaurant", title="search query", description="The query used to search for local entities.")
    type: str = Field(default="EatDrink", description="The specified types used to filter the local entities")

    def exec(self):
        query_params = {
            "q": self.query,
            "type": self.type,
            "userLocation": self.user_location,
            "key": BING_MAPS_API_KEY
        }

        URL = "https://dev.virtualearth.net/REST/v1/LocalSearch/"
        response = requests.get(url=URL, params=query_params)
        return response.json()["resourceSets"][0]["resources"][0]
    

class RouteSearch(BaseModel):
    """ route search
    
    Returned by Routes URL contains a Route resource.
    The information provided in the Route resource includes the route distance, the travel time, and the itinerary details for each route leg.
    For more information: https://learn.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-route
    """

    waypoints: list[str] = Field(description="Specifies two or more locations that define the route and that are in sequential order.")
    travel_mode: str = Field(default="Transit")

    def exec(self):
        waypoints = {
            f"waypoint.{i}": v for i, v in enumerate(self.waypoints)
        }
        query_params = {
            **waypoints,
            "travelMode": self.travel_mode,
            "key": BING_MAPS_API_KEY
        }
        URL = "http://dev.virtualearth.net/REST/v1/Routes"
        response = requests.get(url=URL, params=query_params)
        return response.json()["resourceSets"][0]["resources"][0]

In [4]:
LocalSearch.model_json_schema()

{'description': 'local search\n\nReturns a list of business entities centered around a location or a geographic region.\nFor more information: https://learn.microsoft.com/en-us/bingmaps/rest-services/locations/local-search',
 'properties': {'user_location': {'description': "Coordinates for the user's location on Earth.",
   'title': 'user location',
   'type': 'string'},
  'query': {'default': 'restaurant',
   'description': 'The query used to search for local entities.',
   'title': 'search query',
   'type': 'string'},
  'type': {'default': 'EatDrink',
   'description': 'The specified types used to filter the local entities',
   'title': 'Type',
   'type': 'string'}},
 'required': ['user_location'],
 'title': 'LocalSearch',
 'type': 'object'}

In [5]:
RouteSearch.model_json_schema()

{'description': 'route search\n\nReturned by Routes URL contains a Route resource.\nThe information provided in the Route resource includes the route distance, the travel time, and the itinerary details for each route leg.\nFor more information: https://learn.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-route',
 'properties': {'waypoints': {'description': 'Specifies two or more locations that define the route and that are in sequential order.',
   'items': {'type': 'string'},
   'title': 'Waypoints',
   'type': 'array'},
  'travel_mode': {'default': 'Transit',
   'title': 'Travel Mode',
   'type': 'string'}},
 'required': ['waypoints'],
 'title': 'RouteSearch',
 'type': 'object'}

## Function callings

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.output_parsers import PydanticToolsParser

user_prompt = """\
- 35.6645201,139.6470065
- 35.6835407,139.5386682
"""

messages = ChatMessagePromptTemplate.format_messages([
    ("system", "You are a genius assistant. Give me the best route to visit the following places considering its opening hour."),
    ("human", "{input}")
])

tools = [LocalSearch, RouteSearch]

llm = ChatOpenAI(model="gpt-3.5-turbo")
llm_with_tools = llm.bind_tools(tools)

output_parser = PydanticToolsParser(tools=tools)

chain = messages | llm_with_tools | output_parser

AttributeError: 'list' object has no attribute 'format'

In [11]:
result = llm_with_tools.invoke(messages)

In [15]:
result.additional_kwargs["tool_calls"]

[{'id': 'call_E9IpEZDl5PTk7Kw6g93g6yWZ',
  'function': {'arguments': '{"user_location": "35.6645201,139.6470065"}',
   'name': 'LocalSearch'},
  'type': 'function'},
 {'id': 'call_Me3YeIU1EiJA6YXKdlfrxLsu',
  'function': {'arguments': '{"user_location": "35.6835407,139.5386682"}',
   'name': 'LocalSearch'},
  'type': 'function'}]